## Создание индекса трасс

In [1]:
import sys
import pandas as pd
import numpy as np

sys.path.append('..')
from geolog import FieldIndex, TraceIndex, SegyFilesIndex, BinsIndex


filename = '/home/egor/Desktop/Noise/D01_Raw_shots.sgy'
index = TraceIndex(segyfile=filename)
index._idf.head()

,trace_number,field_id
trace_id,,
0,1656,111906
1,1657,111906
2,1655,111906
3,1658,111906
4,1654,111906


## Перевод индекса трасс в индекс полевых записей

In [2]:
index = FieldIndex(index)
index._idf.head()

trace_number
field_id trace_id              
111906   0                 1656
         1                 1657
         2                 1655
         3                 1658
         4                 1654

## Перевод индекса полевых записей в индекс файлов

In [3]:
paths_dict = dict(zip(index._idf.index.levels[0], [filename]*len(index._idf.index.levels[0])))
index = SegyFilesIndex(index, paths_dict=paths_dict)
index._idf.head()

trace_number
file_id                                    field_id trace_id              
/home/egor/Desktop/Noise/D01_Raw_shots.sgy 111906   0                 1656
                                                    1                 1657
                                                    2                 1655
                                                    3                 1658
                                                    4                 1654

## Перевод индекса файлов в индекс трасс

In [4]:
index = TraceIndex(index)
index._idf.head()

,file_id,field_id,trace_number
trace_id,,,
0,/home/egor/Desktop/Noise/D01_Raw_shots.sgy,111906,1656
1,/home/egor/Desktop/Noise/D01_Raw_shots.sgy,111906,1657
2,/home/egor/Desktop/Noise/D01_Raw_shots.sgy,111906,1655
3,/home/egor/Desktop/Noise/D01_Raw_shots.sgy,111906,1658
4,/home/egor/Desktop/Noise/D01_Raw_shots.sgy,111906,1654


## Создание индекса 2D бинов

In [5]:
dfs = pd.read_csv("/home/egor/Desktop/2TB_SEGY/Sps_out/z400full.Scsv")
dfr = pd.read_csv("/home/egor/Desktop/2TB_SEGY/Sps_out/z400full.Rcsv")
dfx = pd.read_csv("/home/egor/Desktop/2TB_SEGY/Sps_out/z400full.Xcsv").sample(frac=1, random_state=0)[:1000]

bin_size = 2000

index = BinsIndex(dfr=dfr, dfs=dfs, dfx=dfx, bin_size=(bin_size, bin_size), iters=10)
index._idf.head()

,,sline,sid,rline,rid,trace_number,x_s,y_s,z_s,x_r,y_r,z_r,x_m,y_m,az,offset,field_id
bin_id,trace_id,,,,,,,,,,,,,,,,
7/14,0,2617,5387,5281,2517,201,399075.0,5721010.5,148.6,394100.0,5718348.0,169.2,396587.50,5719679.25,-2.650202,5642.652856,1/67081
7/12,1,2469,5369,5281,2517,749,391675.0,5720560.5,153.6,394100.0,5718348.0,169.2,392887.50,5719454.25,-0.739608,3282.648511,1/48897
7/14,2,2617,5387,5281,2518,202,399075.0,5721010.5,148.6,394150.0,5718348.0,168.5,396612.50,5719679.25,-2.645988,5598.618691,1/67081
7/12,3,2469,5369,5281,2518,750,391675.0,5720560.5,153.6,394150.0,5718348.0,168.5,392912.50,5719454.25,-0.729457,3319.756203,1/48897
7/14,4,2617,5387,5281,2519,203,399075.0,5721010.5,148.6,394200.1,5718348.0,168.5,396637.55,5719679.25,-2.641699,5554.597759,1/67081


## Перевод индекса 2D бинов в индекс трасс

In [6]:
index = TraceIndex(index)
index._idf.head()

,bin_id,sline,sid,rline,rid,trace_number,x_s,y_s,z_s,x_r,y_r,z_r,x_m,y_m,az,offset,field_id
trace_id,,,,,,,,,,,,,,,,,
0,7/14,2617,5387,5281,2517,201,399075.0,5721010.5,148.6,394100.0,5718348.0,169.2,396587.50,5719679.25,-2.650202,5642.652856,1/67081
1,7/12,2469,5369,5281,2517,749,391675.0,5720560.5,153.6,394100.0,5718348.0,169.2,392887.50,5719454.25,-0.739608,3282.648511,1/48897
2,7/14,2617,5387,5281,2518,202,399075.0,5721010.5,148.6,394150.0,5718348.0,168.5,396612.50,5719679.25,-2.645988,5598.618691,1/67081
3,7/12,2469,5369,5281,2518,750,391675.0,5720560.5,153.6,394150.0,5718348.0,168.5,392912.50,5719454.25,-0.729457,3319.756203,1/48897
4,7/14,2617,5387,5281,2519,203,399075.0,5721010.5,148.6,394200.1,5718348.0,168.5,396637.55,5719679.25,-2.641699,5554.597759,1/67081
